### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Check data
purchase_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


## Player Count

* Display the total number of players


In [2]:
# checking to see if there is any duplicates in player (SN) column
boolean = any(purchase_data['SN'].duplicated())
print(f"There is duplicated players in SN column.")

# getting number of players by creating a unique array of players, dropping duplicates
player_count = len(purchase_data["SN"].unique())

# putting player counts into a dataframe
total_players = pd.DataFrame({'Total Players':[player_count]})
total_players

There is duplicated players in SN column.


,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# getting number of unique items
unique_items = len(purchase_data["Item ID"].unique())

# calculating average purchase price
average_price = purchase_data['Price'].mean()

# getting total number of purchases
purchase_count = purchase_data['Purchase ID'].count()

# calculating total revenue
total_revenue = purchase_data['Price'].sum()

# putting all values into a dataframe
summary_table = pd.DataFrame({'Number of Unique Items': [unique_items],
                             'Average Price': [average_price],
                             'Number of Purchases': [purchase_count],
                             'Total Revenue': [total_revenue]})

summary_table["Average Price"] = summary_table["Average Price"].astype(float).map(
    "${:,.2f}".format)

summary_table["Total Revenue"] = summary_table["Total Revenue"].astype(float).map(
    "${:,.2f}".format)

summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# creating a new dataframe with only SN and Gender columns
columns = ["SN", "Gender"]
unique_purchasers_df = purchase_data.loc[:,columns]

# removing duplicates based on SN column from the new dataframe
unique_purchasers_df.drop_duplicates(subset ="SN", 
                     keep = 'first', inplace = True) 

# getting the counts and percentages by gender
gender = unique_purchasers_df['Gender']
gender_counts = gender.value_counts()
gender_percent = gender.value_counts(normalize=True)

# putting the counts and percentages into a dataframe
gender_percent = gender_percent.mul(100).round(2).astype(str) + '%'
unique_gender_count = pd.DataFrame({'Total Count': gender_counts, 'Percentage of Players': gender_percent})
unique_gender_count.index.name='Gender'

# sort dataframe in descending percentage of players
unique_gender_count = unique_gender_count.sort_values(['Percentage of Players'],ascending=False)
unique_gender_count

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# group by gender
groupby_gender = purchase_data.groupby("Gender")

# calculating the purchase count and creating the dataframe with the counts. Rename the column to reflect the counts
purchase_count = groupby_gender[["Purchase ID"]].count()
purchase_count = purchase_count.rename(columns={"Purchase ID":"Purchase Count"})

# calculating the average purchase price and creating the dataframe. Rename the column to reflect the average purchase price
avg_price = groupby_gender[["Price"]].mean()
avg_price = avg_price.rename(columns={"Price":"Average Purchase Price"})

# calculating the total purchase value and creating the dataframe with the total purchase value
# rename the column to reflect the total purchase value
total_purchase = groupby_gender[["Price"]].sum()
total_purchase = total_purchase.rename(columns={"Price":"Total Purchase Value"})

# putting the values in the dataframe
gender_summary = pd.merge(purchase_count,avg_price, on="Gender")
gender_summary = pd.merge(gender_summary,total_purchase, on="Gender")

# calculating the average total purchase per individual
gender_summary['Average Total Purchase per Person'] = (gender_summary['Total Purchase Value']/
                                                      unique_gender_count['Total Count'])

# Format the columns
gender_summary["Average Purchase Price"] = gender_summary["Average Purchase Price"].astype(float).map(
    "${:,.2f}".format)

gender_summary["Total Purchase Value"] = gender_summary["Total Purchase Value"].astype(float).map(
    "${:,.2f}".format)

gender_summary["Average Total Purchase per Person"] = gender_summary["Average Total Purchase per Person"].astype(float).map(
    "${:,.2f}".format)

gender_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# creating a new dataframe with only the SN and Gender columns
columns_SN_gender = ["SN", "Age"]
unique_purchasers_df2 = purchase_data.loc[:,columns_SN_gender]

# removing SN duplicates from the new dataframe
unique_purchasers_df2.drop_duplicates(subset ="SN", 
                     keep = 'first', inplace = True) 

# checking the age of the oldest purchaser
print(f"Oldest purchaser is {unique_purchasers_df2['Age'].max()} years old")

# creating the bins for different age groups
age_bins = [0,9,14,19,24,29,34,39,50]

# creating a list of the age group labels
age_grp = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# adding the age groups to the purchase data frame/categorizing the players into the respective age group
unique_purchasers_df2["Age Groups"] = pd.cut(unique_purchasers_df2["Age"], age_bins, labels=age_grp)

# getting the counts and percentages
age = unique_purchasers_df2['Age Groups']
age_counts = age.value_counts()
age_percent = age.value_counts(normalize=True)

# putting into a dataframe
age_percent = age_percent.mul(100).round(2).astype(str) + '%'
unique_age_count = pd.DataFrame({'Total Count': age_counts, 'Percentage of Players': age_percent})
unique_age_count.index.name='Age Group'
unique_age_count = unique_age_count.reindex(["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])
unique_age_count

Oldest purchaser is 45 years old


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# creating the bins for different age groups
age_bins = [0,9,14,19,24,29,34,39,np.inf]

# creating a list of the age group labels
age_grp = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# adding the age groups to the purchase data frame/categorizing the players into the respective age group
purchase_data["Age Groups"] = pd.cut(purchase_data["Age"], age_bins, labels=age_grp)

# group by age
groupby_age = purchase_data.groupby("Age Groups")

# calculating the purchase count and creating the dataframe with the counts. Rename the column to reflect the counts
purchase_count_byAge = groupby_age[["Purchase ID"]].count()
purchase_count_byAge = purchase_count_byAge.rename(columns={"Purchase ID":"Purchase Count"})

# calculating the average purchase price and creating the dataframe. Rename the column to reflect the average price
avg_price_byAge = groupby_age[["Price"]].mean()
avg_price_byAge = avg_price_byAge.rename(columns={"Price":"Average Purchase Price"})

# calculating the total purchase value and creating the dataframe with the value. Rename the column to reflect the value
total_purchase_byAge = groupby_age[["Price"]].sum()
total_purchase_byAge = total_purchase_byAge.rename(columns={"Price":"Total Purchase Value"})

# putting the values in the dataframe
age_summary = pd.merge(purchase_count_byAge,avg_price_byAge, on="Age Groups")
age_summary = pd.merge(age_summary,total_purchase_byAge, on="Age Groups")

# calculating the average total purchase per individual and adding a column to the dataframe
age_summary['Average Total Purchase per Person'] = (age_summary['Total Purchase Value']/
                                                      unique_age_count['Total Count'])

# Format the columns
age_summary["Average Purchase Price"] = age_summary["Average Purchase Price"].astype(float).map(
    "${:,.2f}".format)

age_summary["Total Purchase Value"] = age_summary["Total Purchase Value"].astype(float).map(
    "${:,.2f}".format)

age_summary["Average Total Purchase per Person"] = age_summary["Average Total Purchase per Person"].astype(float).map(
    "${:,.2f}".format)

age_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# group by spender
groupby_spender = purchase_data.groupby("SN")

# calculating the purchase count and creating the dataframe with the counts. Rename the column to reflect the counts
purchase_count_bySpender = groupby_spender[["Purchase ID"]].count()
purchase_count_bySpender = purchase_count_bySpender.rename(columns={"Purchase ID":"Purchase Count"})

# calculating the average purchase price and creating the dataframe. Rename the column to reflect the average purchase price
avg_price_bySpender = groupby_spender[["Price"]].mean()
avg_price_bySpender = avg_price_bySpender.rename(columns={"Price":"Average Purchase Price"})

# putting the values in the dataframe
spender_summary = pd.merge(purchase_count_bySpender,avg_price_bySpender, on="SN")

# calculating the total purchase value and adding a column with the value
spender_summary['Total Purchase Value'] = ((spender_summary['Purchase Count'])*
                                           (spender_summary['Average Purchase Price']))

# sort in descending total purchase value
spender_summary = spender_summary.sort_values(['Total Purchase Value'],ascending=False)

# Format the purchase prices columns
spender_summary["Average Purchase Price"] = spender_summary["Average Purchase Price"].astype(float).map(
    "${:,.2f}".format)

spender_summary["Total Purchase Value"] = spender_summary["Total Purchase Value"].astype(float).map(
    "${:,.2f}".format)

spender_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# retrieve item ID, item name, item price, and purchase id columns
item_columns = ['Item ID','Item Name','Price', 'Purchase ID']
item_table = purchase_data.loc[:,item_columns]

# group by item id and item name
item_identifier = ['Item ID','Item Name']
groupby_item = item_table.groupby(item_identifier)

# calculating the purchase count and creating the dataframe with the counts. Rename the column to reflect the counts
purchase_count_byItem = groupby_item[["Purchase ID"]].count()
purchase_count_byItem = purchase_count_byItem.rename(columns={"Purchase ID":"Purchase Count"})

# sorting the dataframe by descending purchase count
purchase_count_byItem = purchase_count_byItem.sort_values(['Purchase Count'],ascending=False)

# calculating the average purchase price and creating the dataframe. Rename the column to reflect the average price
avg_price_byItem = groupby_item[["Price"]].mean()
avg_price_byItem = avg_price_byItem.rename(columns={"Price":"Item Price"})

# putting the values in the dataframe
merge_variable = ['Item ID','Item Name']
item_summary = pd.merge(purchase_count_byItem,avg_price_byItem, on=merge_variable)

# calculating the total purchase value and adding a column with the value
item_summary['Total Purchase Value'] = ((purchase_count_byItem['Purchase Count'])*
                        (avg_price_byItem['Item Price']))

# sort in descending purchase count
item_summary = item_summary.sort_values(['Purchase Count'],ascending=[False])

# duplicating the table for the next section (most profitable items) to avoid creating the dataframe from scratch 
# after formatting
item_summary_copy = item_summary.copy()

# Format the columns
item_summary["Item Price"] = item_summary["Item Price"].astype(float).map(
    "${:,.2f}".format)

item_summary["Total Purchase Value"] = item_summary["Total Purchase Value"].astype(float).map(
    "${:,.2f}".format)

item_summary.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
60,Wolf,8,$3.54,$28.32


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# sort in descending total purchase value of the duplicated item dataframe
item_summary_copy = item_summary_copy.sort_values(['Total Purchase Value'],ascending=[False])

# Format the purchase prices columns
item_summary_copy["Item Price"] = item_summary_copy["Item Price"].astype(float).map(
    "${:,.2f}".format)

item_summary_copy["Total Purchase Value"] = item_summary_copy["Total Purchase Value"].astype(float).map(
    "${:,.2f}".format)

item_summary_copy.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [11]:
# pulling together some values to form a summary of some observations

# getting the total number of players purchasing at least 1 item
players_total = total_players.iloc[0][0]

# getting the gender that made the most number of purchase, determined by percentage of total purchasers
purchase_gender_list = unique_gender_count.head()
most_purchase_byGender = (list(purchase_gender_list.index.values))[0]
gender_purchase_percent = unique_gender_count.iloc[0]['Percentage of Players']

# getting the top spender and his/her total purchase value
top_spender_list = spender_summary.head()
top_spender = (list(top_spender_list.index.values))[0]
top_spender_total_purchase = spender_summary.iloc[0]['Total Purchase Value']

# getting the most popular item
popular_item_list = item_summary.head()
popular_item_ID_Name = (list(popular_item_list.index.values))[0]
popular_item = popular_item_ID_Name[1]

# getting most profitable item
profitable_item_list= item_summary_copy.head()
profitable_item_ID_Name = (list(profitable_item_list.index.values))[0]
profitable_item = profitable_item_ID_Name[1]

# printing some text that summarize the total number of purchasers, gender group that made the most purchase,
# the top spender with his/her total purchase value, the most popular item and most profitable item 
print(f"There is a total of {players_total} players with {gender_purchase_percent} being {most_purchase_byGender} players.")
print(f"{top_spender} spent the most with a total purchase value of {top_spender_total_purchase}.")
print(f"The most popular item is {popular_item}.")

if profitable_item == popular_item:
    print(f"{profitable_item} is also the most profitable item!")
else:
    print(f"{profitable_item} is the most profitable item.")

There is a total of 576 players with 84.03% being Male players.
Lisosia93 spent the most with a total purchase value of $18.96.
The most popular item is Oathbreaker, Last Hope of the Breaking Storm.
Oathbreaker, Last Hope of the Breaking Storm is also the most profitable item!
